In [1]:
#
import os
import sys
import warnings
import copy
from pathlib import Path
from typing import Union, List

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# Stable Baseline3
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

#
import matplotlib
import pandas as pd
from finrl import config_tickers

#
from rl.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from rl.envs.StockTradingEnv import StockTradingEnv
from rl.experiments._1_same_bigger_data_fundamental.train import (
    CustomDRLAgent,
    get_env_kwargs,
    base_cols,
    data_cols,
    ratios_cols,
)
from rl.experiments.common.utils import init_program
from rl.experiments.common.utils import ignore_warnings
from rl.experiments.common.classes import LearnedAlgorithm

#
from configuration.settings import ProjectDir, ExperimentDir
from rl.experiments._2_ta_orig.Env import StockPortfolioEnv
from rl.experiments._2_ta_orig.train import base_cols, data_cols, features_cols
from rl.experiments.common.classes import Program
from rl.experiments.common.utils import ignore_warnings
from rl.data.CompanyInfo import CompanyInfo

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
ignore_warnings()

In [3]:
_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATE_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"

In [4]:
program = init_program()

In [5]:
#
program = Program(
    prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
    exp_dir=ExperimentDir(root=Path(os.getcwd())),
    DEBUG=False,
)
program.exp_dir.check_and_create_dirs()

#
TICKERS = copy.deepcopy(config_tickers.DOW_30_TICKER)
TICKERS.remove("DOW")  # TODO: I don't have all necessary data for "DOW"

In [6]:
print(TICKERS)
print(program.prj_dir.root)
print(program.exp_dir.root)
print(base_cols)
print(data_cols)
print(features_cols)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS']
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/src/rl/experiments/_2_ta_orig
['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [7]:
# LearnedAlgorithm.__init__.__code__.co_varnames

In [8]:
def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = []
for filepath in program.exp_dir.out.algorithms.glob("*"):
    learned_algorithms.append(get_algorithm(filepath))
    break

In [9]:
# Test Learned Algorithms
print(len(learned_algorithms))
for algo in learned_algorithms:
    env_kwargs = get_env_kwargs(program.dataset)
    env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)
    algo.df_account_value, algo.df_actions = CustomDRLAgent.DRL_prediction(
        model=algo.learned_algorithm, environment=env_gym
    )
# algos_prediction = predicate(program, learned_algorithms)

0


In [10]:
# print(learned_algorithms[0].df_actions)
# print(learned_algorithms[0].df_account_value)

In [11]:
metric = "Annual return"

for algo in learned_algorithms:
    perf_stats_all = backtest_stats(account_value=algo.df_account_value)
    algo.perf_stats_all = pd.DataFrame(perf_stats_all)

In [12]:
def print_learned_algo(algos: List[LearnedAlgorithm]):
    for i in algos:
        try:
            print(i.perf_stats_all.loc[metric][0])
        except Exception as e:
            print(f"{e}")


print_learned_algo(learned_algorithms)

In [13]:
sorted_list = sorted(learned_algorithms, key=lambda x: x.perf_stats_all.loc[metric][0])

In [14]:
# print_learned_algo(sorted_list)
# print(f"{sorted_list[len(sorted_list)].filename}")

best_idx = len(sorted_list) - 1
worst_idk = 0


def get_annual_return(algo: LearnedAlgorithm):
    return algo.perf_stats_all.loc[metric][0]


print(f"{get_annual_return(sorted_list[best_idx])}:  {sorted_list[best_idx].filename.name}")
print(f"{get_annual_return(sorted_list[worst_idk])}:  {sorted_list[worst_idk].filename.name}")

IndexError: list index out of range

In [ ]:
stats = backtest_stats(sorted_list[best_idx].df_account_value)
stats

In [ ]:
start = program.dataset["date"].min()
end = program.dataset["date"].max()
print(f"Start: {start}  -  End: {end}")

In [ ]:
# Baseline
baseline_df = get_baseline("^DJI", start, end)

In [ ]:
baseline_df

In [ ]:
print(baseline_df.index.min())
print(baseline_df.index.max())

In [ ]:
stats = backtest_stats(baseline_df, value_col_name="close")

stats

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 6)
backtest_plot(baseline_df, value_col_name="close")